In [157]:
import pandas as pd


In [158]:
data = pd.read_csv(
    "resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
data.head()


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [159]:
# Remove redundant attributes - block, flat_model, lease_commence_date
data = data.drop(['block', 'street_name', 'flat_model',
                 'lease_commence_date'], axis=1)
data.head()


,month,town,flat_type,storey_range,floor_area_sqm,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,10 TO 12,44.0,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,04 TO 06,68.0,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,01 TO 03,67.0,62 years 05 months,265000.0


In [160]:
# Data transformation - month to year_sold & month_sold
date = pd.to_datetime(data['month'])
data['year_sold'] = date.dt.year
data['month_sold'] = date.dt.month
data = data.drop(['month'], axis=1)
data[['year_sold', 'month_sold']].head()


,year_sold,month_sold
0,2017,1
1,2017,1
2,2017,1
3,2017,1
4,2017,1


In [161]:
print(data['town'].unique())
data['town'].value_counts()


['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'BUKIT TIMAH' 'CENTRAL AREA' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG'
 'HOUGANG' 'JURONG EAST' 'JURONG WEST' 'KALLANG/WHAMPOA' 'MARINE PARADE'
 'PASIR RIS' 'PUNGGOL' 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON'
 'TAMPINES' 'TOA PAYOH' 'WOODLANDS' 'YISHUN']


SENGKANG           12584
PUNGGOL            11032
WOODLANDS          10425
YISHUN             10251
TAMPINES           10072
JURONG WEST         9903
BEDOK               8068
HOUGANG             7346
CHOA CHU KANG       6874
ANG MO KIO          6250
BUKIT MERAH         5740
BUKIT PANJANG       5722
BUKIT BATOK         5049
TOA PAYOH           4793
PASIR RIS           4611
KALLANG/WHAMPOA     4495
QUEENSTOWN          4262
GEYLANG             3729
SEMBAWANG           3726
CLEMENTI            3436
JURONG EAST         3123
BISHAN              2837
SERANGOON           2785
CENTRAL AREA        1246
MARINE PARADE        955
BUKIT TIMAH          389
Name: town, dtype: int64

In [162]:
# Classification based on region
region = {
    'ANG MO KIO': 'CENTRAL',
    'BEDOK': 'EAST',
    'BISHAN': 'CENTRAL',
    'BUKIT BATOK': 'WEST',
    'BUKIT MERAH': 'SOUTH',
    'BUKIT PANJANG': 'WEST',
    'BUKIT TIMAH': 'SOUTH',
    'CENTRAL AREA': 'CENTRAL',
    'CHOA CHU KANG': 'WEST',
    'CLEMENTI': 'WEST',
    'GEYLANG': 'EAST',
    'HOUGANG': 'EAST',
    'JURONG EAST': 'WEST',
    'JURONG WEST': 'WEST',
    'KALLANG/WHAMPOA': 'EAST',
    'MARINE PARADE': 'EAST',
    'PASIR RIS': 'EAST',
    'PUNGGOL': 'EAST',
    'QUEENSTOWN': 'SOUTH',
    'SEMBAWANG': 'NORTH',
    'SENGKANG': 'NORTH',
    'SERANGOON': 'EAST',
    'TAMPINES': 'EAST',
    'TOA PAYOH': 'CENTRAL',
    'WOODLANDS': 'NORTH',
    'YISHUN': 'NORTH'
}
data['region'] = data['town'].transform(lambda t: region[t])
data = data.drop(['town'], axis=1)
data['region'].value_counts()


EAST       53093
NORTH      36986
WEST       34107
CENTRAL    15126
SOUTH      10391
Name: region, dtype: int64

In [163]:
data['flat_type'].value_counts()


4 ROOM              62569
5 ROOM              37845
3 ROOM              35241
EXECUTIVE           11570
2 ROOM               2347
MULTI-GENERATION       72
1 ROOM                 59
Name: flat_type, dtype: int64

In [164]:
# Remove rows with arbitrary room numbers
data = data.loc[data['flat_type'] != 'MULTI-GENERATION']


In [165]:
# Data transformation - flat_type to room_num
roomNum = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6
}
data['flat_type'] = data['flat_type'].transform(lambda ft: roomNum[ft])
data = data.rename(columns={'flat_type': 'room_num'})
data[['room_num']].head()


,room_num
0,2
1,3
2,3
3,3
4,3


In [166]:
data['storey_range'].value_counts()


04 TO 06    34479
07 TO 09    31345
10 TO 12    27878
01 TO 03    26468
13 TO 15    14319
16 TO 18     6733
19 TO 21     2860
22 TO 24     2090
25 TO 27     1241
28 TO 30      788
31 TO 33      419
34 TO 36      394
37 TO 39      351
40 TO 42      171
43 TO 45       48
46 TO 48       34
49 TO 51       13
Name: storey_range, dtype: int64

In [167]:
# Data transformation - storey_range to storey_avg
def getAvgStorey(sr):
    [lower, upper] = [int(s) for s in sr.split(" TO ")]
    avg = int((lower + upper) / 2)
    return avg


data['storey_range'] = data['storey_range'].transform(getAvgStorey)
data = data.rename(columns={'storey_range': 'storey_avg'})
data[['storey_avg']].head()


,storey_avg
0,11
1,2
2,2
3,5
4,2


In [168]:
data['remaining_lease'].value_counts()


94 years 10 months    1366
94 years 09 months    1341
94 years 11 months    1320
94 years 08 months    1185
95 years              1144
                      ... 
97 years 07 months       1
97 years 09 months       1
97 years 01 month        1
93 years 0 months        1
42 years 10 months       1
Name: remaining_lease, Length: 663, dtype: int64

In [169]:
# Data transformation - remaining_lease - integers in year & month to floating numbers in year
def getRemainingLeaseInYears(rl):
    rl = rl.split()
    y, m = int(rl[0]), int(rl[2]) if len(rl) > 2 else 0
    years = y + m / 12
    return "{:.2f}".format(years)


data['remaining_lease'] = data['remaining_lease'].transform(
    getRemainingLeaseInYears)
data[['remaining_lease']].head()


,remaining_lease
0,61.33
1,60.58
2,62.42
3,62.08
4,62.42


In [170]:
# Data scaling - resale_price - dollars to 100k dollars
data['resale_price'] = data['resale_price'] / 100000
data = data.rename(columns={'resale_price': 'resale_price_100k'})


In [171]:
data


,room_num,storey_avg,floor_area_sqm,remaining_lease,resale_price_100k,year_sold,month_sold,region
0,2,11,44.0,61.33,2.32,2017,1,CENTRAL
1,3,2,67.0,60.58,2.50,2017,1,CENTRAL
2,3,2,67.0,62.42,2.62,2017,1,CENTRAL
3,3,5,68.0,62.08,2.65,2017,1,CENTRAL
4,3,2,67.0,62.42,2.65,2017,1,CENTRAL
...,...,...,...,...,...,...,...,...
149698,6,11,146.0,64.67,8.00,2023,3,NORTH
149699,6,2,146.0,64.50,8.00,2023,3,NORTH
149700,6,5,142.0,64.25,7.70,2023,3,NORTH
149701,6,11,146.0,63.42,8.18,2023,3,NORTH


In [172]:
data.to_csv('data_cleaned.csv')
